## Libraries (local machine)

In [5]:
from datetime import date, datetime, timedelta
from dotenv import load_dotenv
from os import getenv
from random import randint, random
import numpy as np
import pandas as pd

load_dotenv()

True

## Libraries (Google Colab)

In [ ]:
from datetime import date, datetime, timedelta
from google.colab import userdata
from random import choice, random
import numpy as np
import pandas as pd

def getenv(secretName: str, default_value):
  try:
    return userdata.get(secretName)
  except:
    return default_value

## Classes definition

In [ ]:
#from typing import Any, NewType

class Project:
   def __init__(self, name:str, DF_date: date, MCS_date: date, Pilot_date: date, SOP_date: date):
      self.name = name
      self.important_dates = {
         "Design freeze": DF_date,
         "MCS": MCS_date,
         "Pilot": Pilot_date,
         "SOP": SOP_date
      }

   def __str__(self):
      return self.name

class Item_Master:
   def __init__(self):
      columns = {
        # ECN identification data
        "Project": [],
        "ECN": [],
        "ECN release": [],
        "RFQ date": [],
        # Part number data
        "Part number": [],
        "Complexity": [],
        "EAU": [],  # EAU stands for Estimated Annual Use
        # Supplier data
        "Delivery profile": [],
        "Quotation profile": [],
        "Price profile": [],
        "Punctuality profile": [],
        "Supplier ID": [],
        "Supplier name": [],
        "Quotation date": [],
        "Price": [],
        "Lead time": [],
        # Sample delivery data
        "ETA": [],
        "Delivery date": [],
        "ISIR documents": [],
        # Environment data
        "REQ date": [],
        "PO date": [],
        "ISIR approval": [],
        "PPAP approval": [],
        "Contract date": [],
        # Fuzzy inputs
        "Quotation time": [],
        "OTD": [],
        "Delivery time": [],
        # Other information
        "FY Spend": [], # EAU * Price
        "Awarded": [], # bool (False only for quotations, True for awarded business)
        # Readiness
        "MCS ready": [],
        "Pilot ready": [],
        "SOP ready": []
      }

      self.df = pd.DataFrame(columns)

class Part_Number:
    def __init__(self, pn: str, complexity: str, eau: int):
        self.pn = pn
        self.complexity = complexity
        self.eau = eau  # I am using an integer because most of the materials have EA as UOM, with limited exceptions.

    def __str__(self):
        return self.pn

class ECN:
    def __init__(self, project: Project, ecn_id: str, ecn_date: date, pn_list: list[Part_Number]):
        self.project = project
        self.ecn_id = ecn_id
        self.ecn_date = ecn_date
        self.items = pn_list
        self.quotations = []

        self.readiness = {
            "MCS ready": False,
            "Pilot ready": False,
            "SOP ready": False
        }

    def __str__(self):
        return self.ecn_id

    def display_as_df(self):
      df_layout = {
          "Project": [],
          "ECN": [],
          "ECN release": [],
          "Part number": [],
          "Complexity": [],
          "EUA": []
       }

      df = pd.DataFrame(df_layout)

      for item in self.items:
        df.loc[len(df)] = [self.project.name, self.ecn_id, self.ecn_date, item.pn, item.complexity, item.eau]

      return df

class Quotation:
    def __init__(self, ecn: ECN, supplier: object, date: date):
        self.ecn = ecn
        self.supplier = supplier
        self.date = date
        self.awarded = False

        self.ecn.quotations.append(self)

        columns = {
        # ECN identification data
        "Project": [],
        "ECN": [],
        "ECN release": [],
        "RFQ date": [],
        # Part number data
        "Part number": [],
        "Complexity": [],
        "EAU": [],  # EAU stands for Estimated Annual Use
        # Supplier data
        "Delivery profile": [],
        "Quotation profile": [],
        "Price profile": [],
        "Punctuality profile": [],
        "Supplier ID": [],
        "Supplier name": [],
        "Quotation date": [],
        "Price": [],
        "Lead time": [],
        # Fuzzy inputs
        "Quotation time": [],
        # Other information
        "FY Spend": [], # EAU * Price
        "Awarded": [] # bool (False only for quotations, True for awarded business)
        }

        self.df = pd.DataFrame(columns)

class Supplier:
    def __init__(self, id: str | int, name: str, price_profile: str = "regular", quotation_profile: str = "regular", punctuality_profile: str = "regular", delivery_profile: str = "regular"):
        self.id = self.__check_id(id)
        self.name = name
        self.quotations = []
        self.awarded_quotations = []

        self.delivery_profile = delivery_profile
        self.quotation_profile = quotation_profile
        self.price_profile = price_profile
        self.punctuality_profile = punctuality_profile

        price_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (0.85, 0.85),
          "regular": (1, 1),
          "high": (1.2, 1.1)
        }

        quotation_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (28.975, 25.1133753461483),
          "regular": (27.7241379310345, 21.5974276436511),
          "high": (24.9444444444444, 10.258266234788)
        }

        punctuality_profile_map = { # Probability
          "low": 0.19047619047619,
          "regular": 0.473684210526316,
          "high": 0.638888888888889
        }

        self.ETA_difference = {
            "punctual": (0.888888888888889, 1.01273936708367),
            "unpunctual": (4.24137931034483, 2.69463981708917)
        }

        delivery_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (0.8, 0.8),
          "regular": (1, 1),
          "high": (1, 1.3)
        }

        µ_price_profile_factor, σ_price_profile_factor = price_profile_map[price_profile]
        µ_delivey_profile_factor, σ_delivery_profile_factor = delivery_profile_map[delivery_profile]

        self.price_complexity_map = {
            "high": (float(getenv("AVG_PRICE_HIGH_COMPLEXITY", 0)) * µ_price_profile_factor, float(getenv("STDEV_PRICE_HIGH_COMPLEXITY", 1)) * σ_price_profile_factor),
            "medium": (float(getenv("AVG_PRICE_MEDIUM_COMPLEXITY", 0)) * µ_price_profile_factor, float(getenv("STDEV_PRICE_MEDIUM_COMPLEXITY", 1)) * σ_price_profile_factor),
            "low": (float(getenv("AVG_PRICE_LOW_COMPLEXITY", 0)) * µ_price_profile_factor, float(getenv("STDEV_PRICE_LOW_COMPLEXITY", 1)) * σ_price_profile_factor),
            "minimum": float(getenv("MINIMUM_PRICE", 0)) * µ_price_profile_factor
        }

        self.µ_quotation_time, self.σ_quotation_time = quotation_profile_map[quotation_profile]
        self.minimum_quotation_time = 9

        self.µ_delivery_time = 34.6206896551724 * µ_delivey_profile_factor
        self.σ_delivery_time = 16.2802512871323 * σ_delivery_profile_factor
        self.minimum_delivery_time = 12 * µ_delivey_profile_factor

        self.µ_isir_documents_upload = 0.348484848484849
        self.σ_isir_documents_upload = 0.936317241478537

        self.punctual_p = punctuality_profile_map[punctuality_profile]

    def __str__(self):
        return self.name

    def __check_id(self, id: str | int):
        if len(str(id)) < 8 or len(str(id)) > 8:
          raise Exception("Invalid supplier ID")
        else:
          return str(id)

    def quote(self, ecn: ECN, rfq_date: date, lead_time: int = 0):
        not_quoted_yet = True
        for quotation in self.quotations:
            if quotation.ecn.ecn_id == ecn.ecn_id:
                print(f"{self.name} already quoted {ecn.ecn_id}.")
                not_quoted_yet = False

        if not_quoted_yet:
          min_price = self.price_complexity_map["minimum"]

          quotation_time = max(round(np.random.normal(self.µ_quotation_time, self.σ_quotation_time)), self.minimum_quotation_time)
          quotation_date = rfq_date + timedelta(days=quotation_time)

          quotation = Quotation(ecn, self, quotation_date)

          for part_number in ecn.items:
            complexity = part_number.complexity
            µ, σ = self.price_complexity_map[complexity]
            price = round(max(np.random.normal(µ, σ), min_price), 2)
            spend = part_number.eau * price

            if lead_time == 0:
              lt = np.nan
            elif lead_time > 0:
              lt = lead_time
            else:
              raise Exception("Lead time cannot be less than 1 day.")

            quotation.df.loc[len(quotation.df)] = [ecn.project.name, ecn.ecn_id, ecn.ecn_date, rfq_date, part_number.pn, complexity, part_number.eau, self.delivery_profile, self.quotation_profile, self.price_profile, self.punctuality_profile, self.id, self.name, quotation_date, price, lt, quotation_time, spend, False]

          self.quotations.append(quotation)
          return quotation.df

In [7]:
class Environment:
  def __init__(self):
    self.suppliers = []
    self.ecns = []

    self.item_master = Item_Master().df

    self.part_kinds = {
        "A": {
            "average": float(getenv("AVG_A_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_A_PART_KIND", 1)),
            "complexity": {"low": 0.6818181818182, "medium": 0.318181818181818, "high": 0},
            "parts": []
        },
        "B": {
            "average": float(getenv("AVG_B_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_B_PART_KIND", 1)),
            "complexity": {"low": 1/3, "medium": 2/3, "high": 0},
            "parts": []
        },
        "C": {
            "average": float(getenv("AVG_C_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_C_PART_KIND", 1)),
            "complexity": {"low": 1, "medium": 0, "high": 0},
            "parts": []
        },
        "D": {
            "average": float(getenv("AVG_D_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_D_PART_KIND", 1)),
            "complexity": {"low": 0.090909090909090909, "medium": 0.727272727272727, "high": 0.181818181818182},
            "parts": []
        },
        "E": {
            "average": float(getenv("AVG_E_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_E_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "F": {
            "average": float(getenv("AVG_F_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_F_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "G": {
            "average": float(getenv("AVG_G_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_G_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        }
    }

    self.environment_times = {
      "release_ecn": (119.47619047619*0.5, 160.671596446795*0.5625),
      "release_ecn_min": -110,
      "release_ecn_max": 436,
      "send_rfq": (3.0625, 3.98415201798981),
      "create_req": (8.88571428571429, 8.92406428682835),
      "send_po": (7.53409090909091, 3.31471419560376),
      "approve_isir": (26.8, 33.7110446645324),
      "approve_ppap": (2.63559322033898, 5.29741955281872),
      "upload_contract": (2.2156862745098, 3.23234718683581)
    }

    self.µ_eau_qty = 319.395833333333
    self.σ_eau_qty = 364.965095363013
    self.min_eau_qty = 23

  def add_supplier(self, supplier: Supplier):
    self.suppliers.append(supplier)

  def add_suppliers(self, suppliers: list[Supplier]):
    for supplier in suppliers:
      self.add_supplier(supplier)

  def gen_ecns(self, project: Project, qty: int):
    for i in range(qty):
      ecn_part_numbers = []
      ecn_eau = max(round(np.random.normal(loc=self.μ_eau_qty, scale=self.σ_eau_qty)), self.min_eau_qty)

      while len(ecn_part_numbers) == 0:
        for key in self.part_kinds.keys():
          kind_complexity_keys = list(self.part_kinds[key]["complexity"].keys())
          kind_complexity_probabilities = list(self.part_kinds[key]["complexity"].values())

          for j in range(max(int(np.random.normal(self.part_kinds[key]["average"], self.part_kinds[key]["stdev"])), 0)):
            category_part_number = len(self.part_kinds[key]["parts"]) + 1
            complexity = np.random.choice(kind_complexity_keys, p=kind_complexity_probabilities)

            part_number = Part_Number(pn=f"A0{key}{str(category_part_number).zfill(6)}", complexity=complexity, eau=ecn_eau)

            self.part_kinds[key]["parts"].append(part_number)
            ecn_part_numbers.append(part_number)

      ecn_number = len(self.ecns) + 1
      µ_ecn_release_time, σ_ecn_release_time = self.environment_times["release_ecn"]
      ecn_date = project.important_dates["Design freeze"] + timedelta(days=min(max(round(np.random.normal(loc=µ_ecn_release_time, scale=σ_ecn_release_time)), -160), 436))
      self.ecns.append(ECN(project=project, ecn_id=f"ECN{str(ecn_number).zfill(7)}", ecn_date=ecn_date, pn_list=ecn_part_numbers))

  def quote_ecn(self, ecn: ECN):
    µ_rfq_time, σ_rfq_time = self.environment_times["send_rfq"]

    for supplier in self.suppliers:
      rfq_date = ecn.ecn_date + timedelta(days=max(round(np.random.normal(loc=µ_rfq_time, scale=σ_rfq_time)), 0))

      quotation = supplier.quote(ecn, rfq_date)
      self.item_master = pd.concat([self.item_master, quotation], ignore_index=True)

    return self.item_master[self.item_master["ECN"] == ecn.ecn_id]

  def implement_ecn(self, ecn: ECN, awarded_supplier: Supplier):
    for supplier in self.suppliers:
      if supplier != awarded_supplier:
        for quotation in supplier.quotations:
          if quotation.ecn == ecn and quotation.awarded == True:
            raise Exception(f"{ecn.ecn_id} has already been implemented with {supplier.name}.")

    quotation_not_found = True
    working_quotation = None
    for quotation in awarded_supplier.quotations:
      if quotation.ecn == ecn:
        if quotation.awarded == True:
          raise Exception(f"{ecn.ecn_id} has already been implemented with {awarded_supplier.name}.")
        else:
          quotation.awarded = True
          working_quotation = quotation
          quotation_not_found = False
    if quotation_not_found:
      raise Exception(f"{ecn.ecn_id} has not been quoted by {awarded_supplier.name}.")

    µ_req_time, σ_req_time = self.environment_times["create_req"]
    µ_po_time, σ_po_time = self.environment_times["send_po"]
    µ_delivery_time, σ_delivery_time = (awarded_supplier.μ_delivery_time, awarded_supplier.σ_delivery_time)
    µ_documents_time, σ_documents_time = (awarded_supplier.μ_isir_documents_upload, awarded_supplier.σ_isir_documents_upload)
    µ_isir_time, σ_isir_time = self.environment_times["approve_isir"]
    µ_ppap_time, σ_ppap_time = self.environment_times["approve_ppap"]
    µ_contract_time, σ_contract_time = self.environment_times["upload_contract"]

    req_time = max(round(np.random.normal(loc=µ_req_time, scale=σ_req_time)), 0)
    po_time = max(round(np.random.normal(loc=µ_po_time, scale=σ_po_time)), 0)
    eta_time = max(round(np.random.normal(loc=µ_delivery_time, scale=σ_delivery_time)), awarded_supplier.minimum_delivery_time)

    if random() < awarded_supplier.punctual_p:
      µ_eta_difference, σ_eta_difference = awarded_supplier.ETA_difference["punctual"]
      delivery_time = -max(round(np.random.normal(loc=µ_eta_difference, scale=σ_eta_difference)), 0)
      otd = True
    else:
      µ_eta_difference, σ_eta_difference = awarded_supplier.ETA_difference["unpunctual"]
      delivery_time = max(round(np.random.normal(loc=µ_eta_difference, scale=σ_eta_difference)), 1)
      otd = False

    documents_time = max(round(np.random.normal(loc=µ_documents_time, scale=σ_documents_time)), 0)
    isir_time = max(round(np.random.normal(loc=µ_isir_time, scale=σ_isir_time)), 0)
    ppap_time = max(round(np.random.normal(loc=µ_ppap_time, scale=σ_ppap_time)), 0)
    contract_time = max(round(np.random.normal(loc=µ_contract_time, scale=σ_contract_time)), 0)

    req_date = working_quotation.date + timedelta(days=req_time)
    po_date = req_date + timedelta(days=po_time)
    eta_date = po_date + timedelta(days=eta_time)
    delivery_date = eta_date + timedelta(days=delivery_time)
    documents_date = delivery_date + timedelta(days=documents_time)
    isir_date = documents_date + timedelta(days=isir_time)
    ppap_date = isir_date + timedelta(days=ppap_time)
    contract_date = ppap_date + timedelta(days=contract_time)

    if delivery_date <= ecn.project.important_dates["MCS"]:
      mcs_ready = True
    else:
      mcs_ready = False

    if ppap_date <= ecn.project.important_dates["Pilot"]:
      pilot_ready = True
    else:
      pilot_ready = False

    if contract_date <= ecn.project.important_dates["SOP"] - timedelta(weeks=6):
      sop_ready = True
    else:
      sop_ready = False

    self.item_master.loc[(self.item_master["ECN"] == ecn.ecn_id) & (self.item_master["Supplier ID"] == awarded_supplier.id), ["ETA", "Delivery date", "ISIR documents", "Req date", "PO date", "ISIR approval", "PPAP approval", "Contract date", "OTD", "Delivery time", "Awarded", "MCS ready", "Pilot ready", "SOP ready"]] = [eta_date, delivery_date, documents_date, req_date, po_date, isir_date, ppap_date, contract_date, otd, eta_time+delivery_time, True, mcs_ready, pilot_ready, sop_ready]

    return self.item_master[(self.item_master["ECN"] == ecn.ecn_id) & (self.item_master["Supplier ID"] == awarded_supplier.id)]

  def quote_all_ecns(self):
    for ecn in self.ecns:
      self.quote_ecn(ecn)
    return self.item_master

  def gen_initial_item_master_df(self):
    for ecn in self.ecns:
      random_supplier = choice(self.suppliers)

      self.implement_ecn(ecn, random_supplier)

In [8]:
env = Environment()
alaska = Project(name="Alaska", DF_date=date(2024,4,9), MCS_date=date(2024,6,17), Pilot_date=date(2024,8,1), SOP_date=date(2024,11,24))

supplier_1 = Supplier(
    id="10000001",
    name="Tuberías ABC, S.A. de C.V.",
    price_profile="regular",
    quotation_profile="high",
    punctuality_profile="regular",
    delivery_profile="regular"
)
supplier_2 = Supplier(
    id="10000002",
    name="Tuberías DEF, S.A.",
    price_profile="regular",
    quotation_profile="regular",
    punctuality_profile="regular",
    delivery_profile="regular"
)
supplier_3 = Supplier(
    id="10000003",
    name="Tuberías GHI, S.A.S.",
    price_profile="high",
    quotation_profile="low",
    punctuality_profile="regular",
    delivery_profile="regular"
)
supplier_4 = Supplier(
    id="10000004",
    name="Tuberías JKL y asociados, S. en C.",
    price_profile="high",
    quotation_profile="low",
    punctuality_profile="low",
    delivery_profile="regular"
)
supplier_5 = Supplier(
    id="10000005",
    name="Tuberías MNO, S.A.P.I. de C.V.",
    price_profile="low",
    quotation_profile="low",
    punctuality_profile="high",
    delivery_profile="regular"
)

env.add_suppliers([supplier_1, supplier_2, supplier_3, supplier_4, supplier_5])
#env.add_suppliers([supplier_1])

In [9]:
env.gen_ecns(alaska, 10)

In [10]:
env.quote_all_ecns()

,Project,ECN,ECN release,RFQ date,Part number,Complexity,EAU,Supplier ID,Supplier name,Quotation date,...,PPAP approval,Contract date,Quotation time,OTD,Delivery time,FY Spend,Awarded,MCS ready,Pilot ready,SOP ready
0,Alaska,ECN0000001,2024-08-05,2024-08-07,A0B000001,medium,600.0,10000001,"Tuberías ABC, S.A. de C.V.",2024-08-20,...,NaN,NaN,13.0,NaN,NaN,21792.00,0.0,NaN,NaN,NaN
1,Alaska,ECN0000001,2024-08-05,2024-08-07,A0E000001,high,600.0,10000001,"Tuberías ABC, S.A. de C.V.",2024-08-20,...,NaN,NaN,13.0,NaN,NaN,49842.00,0.0,NaN,NaN,NaN
2,Alaska,ECN0000001,2024-08-05,2024-08-13,A0B000001,medium,600.0,10000002,"Tuberías DEF, S.A.",2024-09-12,...,NaN,NaN,30.0,NaN,NaN,21864.00,False,NaN,NaN,NaN
3,Alaska,ECN0000001,2024-08-05,2024-08-13,A0E000001,high,600.0,10000002,"Tuberías DEF, S.A.",2024-09-12,...,NaN,NaN,30.0,NaN,NaN,66612.00,False,NaN,NaN,NaN
4,Alaska,ECN0000001,2024-08-05,2024-08-07,A0B000001,medium,600.0,10000003,"Tuberías GHI, S.A.S.",2024-09-10,...,NaN,NaN,34.0,NaN,NaN,34764.00,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Alaska,ECN0000010,2024-03-15,2024-03-16,A0B000006,medium,256.0,10000001,"Tuberías ABC, S.A. de C.V.",2024-03-29,...,NaN,NaN,13.0,NaN,NaN,9822.72,False,NaN,NaN,NaN
161,Alaska,ECN0000010,2024-03-15,2024-03-17,A0B000006,medium,256.0,10000002,"Tuberías DEF, S.A.",2024-05-07,...,NaN,NaN,51.0,NaN,NaN,18862.08,False,NaN,NaN,NaN
162,Alaska,ECN0000010,2024-03-15,2024-03-19,A0B000006,medium,256.0,10000003,"Tuberías GHI, S.A.S.",2024-05-17,...,NaN,NaN,59.0,NaN,NaN,11535.36,False,NaN,NaN,NaN
163,Alaska,ECN0000010,2024-03-15,2024-03-15,A0B000006,medium,256.0,10000004,"Tuberías JKL y asociados, S. en C.",2024-04-15,...,NaN,NaN,31.0,NaN,NaN,23503.36,False,NaN,NaN,NaN


In [18]:
env.item_master[env.item_master["ECN"] == "ECN0000001"]

,Project,ECN,ECN release,RFQ date,Part number,Complexity,EAU,Supplier ID,Supplier name,Quotation date,...,PPAP approval,Contract date,Quotation time,OTD,Delivery time,FY Spend,Awarded,MCS ready,Pilot ready,SOP ready
0,Alaska,ECN0000001,2024-05-29,2024-06-01,A0D000001,medium,100.0,10000001,"Tuberías ABC, S.A. de C.V.",2024-07-04,...,NaN,NaN,33.0,NaN,NaN,6294.0,0.0,NaN,NaN,NaN
1,Alaska,ECN0000001,2024-05-29,2024-05-30,A0D000001,medium,100.0,10000002,"Tuberías DEF, S.A.",2024-06-18,...,NaN,NaN,19.0,NaN,NaN,193.0,False,NaN,NaN,NaN
2,Alaska,ECN0000001,2024-05-29,2024-05-31,A0D000001,medium,100.0,10000003,"Tuberías GHI, S.A.S.",2024-07-27,...,NaN,NaN,57.0,NaN,NaN,7633.0,False,NaN,NaN,NaN
3,Alaska,ECN0000001,2024-05-29,2024-06-04,A0D000001,medium,100.0,10000004,"Tuberías JKL y asociados, S. en C.",2024-06-27,...,NaN,NaN,23.0,NaN,NaN,4983.0,False,NaN,NaN,NaN
4,Alaska,ECN0000001,2024-05-29,2024-05-29,A0D000001,medium,100.0,10000005,"Tuberías MNO, S.A.P.I. de C.V.",2024-06-27,...,NaN,NaN,29.0,NaN,NaN,5162.0,False,NaN,NaN,NaN


In [19]:
env.implement_ecn(env.ecns[0], supplier_4)

/tmp/ipython-input-2624518232.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-07-27' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.item_master.loc[(self.item_master["ECN"] == ecn.ecn_id) & (self.item_master["Supplier ID"] == awarded_supplier.id), ["ETA", "Delivery date", "ISIR documents", "Req date", "PO date", "ISIR approval", "PPAP approval", "Contract date", "OTD", "Delivery time", "Awarded", "MCS ready", "Pilot ready", "SOP ready"]] = [eta_date, delivery_date, documents_date, req_date, po_date, isir_date, ppap_date, contract_date, otd, eta_time+delivery_time, True, mcs_ready, pilot_ready, sop_ready]
/tmp/ipython-input-2624518232.py:176: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-07-31' has dtype incompatible with float64, please explicitly cast to a c

,Project,ECN,ECN release,RFQ date,Part number,Complexity,EAU,Supplier ID,Supplier name,Quotation date,...,Contract date,Quotation time,OTD,Delivery time,FY Spend,Awarded,MCS ready,Pilot ready,SOP ready,Req date
3,Alaska,ECN0000001,2024-05-29,2024-06-04,A0D000001,medium,100.0,10000004,"Tuberías JKL y asociados, S. en C.",2024-06-27,...,2024-09-26,23.0,False,16.0,4983.0,True,False,False,True,2024-07-04
